# Diverse statistikk frå Canvas web_logs
Eg lager ulike statistikkar basert på Canvas web_logs her; dersom det er "vellukka" kan eg implementere det i den daglege rutina *diverse_statistikk.py*

In [1]:
import pandas as pd
import glob
import os
import pyodbc
import akvut
from datetime import date, timedelta

today = date.today()
idag = today.isoformat()
igår = (today - timedelta(days=1)).isoformat()


Eg sjekker om det ligg inne web_logs-filer frå i dag:

In [2]:

logger = akvut.lag_logger(f'loggfil-diverse_statistikk-{idag}.log')

filer = glob.glob("web_logs-*.txt")


Dersom det gjer det les eg dei inn i ein giga-dataramme:

In [3]:
logger.info(f"Les {len(filer)} filer")
data = pd.concat([pd.read_csv(f) for f in filer])
antal = len(data)
logger.info(f"Antall linjer: {antal}")


2025-01-16 15:52:40,724 - my_logger - INFO - Les 2 filer
2025-01-16 15:53:17,641 - my_logger - INFO - Antall linjer: 2521762


Så kan eg gjere ulike statistikkar.

## Antal klikk på profilbilete

In [8]:
klikk = len(data[data['value.url'].str.contains('/images/thumbnails/')])
andel = klikk/antal
ny_liste = pd.DataFrame([{'dato': idag, 'profilbilete': andel}])
gamal_liste = pd.read_csv("diverse_statistikk.csv")
oppdatert_liste = pd.concat([ny_liste, gamal_liste]).groupby('dato').sum().reset_index()
oppdatert_liste.to_csv("diverse_statistikk.csv", index=False)

## Data til Håvard om kva klikka vert brukt til
Her er vi litt usikre på kva som eigentleg vert vist; vi lager ein tabell og så ser vi over tid kva som står der.

In [4]:
t1 = data[~data['value.url'].str.contains("/api/")]
# utan_user_id = t1[t1['value.user_id'].isna()]
med_user_id = t1[~t1['value.user_id'].isna()]

In [5]:
bruk_desse = med_user_id[['value.timestamp', 'value.user_id', 'value.course_id', 'value.web_application_controller', 'value.web_application_action', 'value.web_application_context_type', 'value.url']]
conn_str = os.environ["Connection_SQL"]
with pyodbc.connect(conn_str) as cnxn:
    with cnxn.cursor() as cursor:
        cursor.fast_executemany = True
        cursor.executemany("INSERT INTO dbo.akv_ny_canvas_history VALUES(?, ?, ?, ?, ?, ?, ?)", bruk_desse.values)

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect); [IM002] [Microsoft][ODBC Driver Manager] Invalid connection string attribute (0)')

Neste steg: finn ut kvifor eg ikkje lengre får kontakt med  databasen frå WSL.

In [ ]:
oppdaterte_lister = pd.concat([nye_lister, gamle_lister]).groupby('dato').sum().reset_index()
oppdaterte_lister.to_csv("diverse_statistikk.csv", index=False)

avsendar = "aasmund.kvamme@hvl.no"
mottakarar = ["aasmund.kvamme@hvl.no"]
tittel = "Diverse statistikk frå web_log"
vedlegg = "diverse_statistikk.csv"
innhald = f"Andel klikk på profilbilete: {klikk/antal*100:.2f} ({klikk} av {antal})"

akvut.send_epost(tittel, innhald, avsendar, mottakarar, vedlegg)
logger.info("Sendt epost til {mottakarar}")